<a href="https://colab.research.google.com/github/DrVenkataRajeshKumar/S12/blob/master/Trail1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install --no-cache-dir git+https://github.com/albu/albumentations > /dev/null && echo "Albumentation library is successfully installed!"
!git clone https://github.com/DrVenkataRajeshKumar/EVA4Library
import sys
sys.path.append("EVA4Library")

  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-4zlmooz_
Albumentation library is successfully installed!
fatal: destination path 'EVA4Library' already exists and is not an empty directory.


In [0]:
import torch
import eva4datatransforms
from eva4albumentations import AlbumentationTransforms
import rn
import eva4modelstats
import eva4modeltrainer
import train_test_loader
import plots
from gradcam import GRADCAM, PLOT
from tinyimagenet import TinyImageNetDataSet

In [12]:
import albumentations as A
import cv2

channel_means = (0.49139968, 0.48215841, 0.44653091)
channel_stdevs = (0.24703223, 0.24348513, 0.26158784)
train_transform = AlbumentationTransforms([       
                                                                
                                      A.HorizontalFlip(p = 0.7),
                                      A.PadIfNeeded(min_height=70, min_width=70, border_mode=4, value=None, mask_value=None, always_apply=False, p=1.0),
                                      A.RGBShift(r_shift_limit=50, g_shift_limit=50, b_shift_limit=50, p=0.5),
                                      A.GaussNoise(var_limit=(10.0, 50.0), mean=0, always_apply=False, p=0.5),
                                      A.RandomCrop(64, 64, always_apply=False, p=1.0),
                                      
                                      A.Rotate(limit=50, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
                                      A.Normalize(mean=channel_means, std=channel_stdevs),
                                      A.Cutout(num_holes=1, max_h_size=32,max_w_size = 32,p=0.5) 
                                       ])
test_transform = AlbumentationTransforms([A.Normalize(mean=channel_means, std=channel_stdevs)])

train_dataset , test_dataset,classes = TinyImageNetDataSet(train_split = 70,test_transforms = test_transform,train_transforms = train_transform)

Images already downloaded...


In [0]:
import eva4dataloaders
dataloader = eva4dataloaders.DataLoader(batch_size=640)

# train dataloader
train_loader = dataloader.load(train_dataset)

# test dataloader
test_loader = dataloader.load(test_dataset)

In [14]:
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = torchvision.models.resnet18(pretrained=False, num_classes=200).to(device)
summary(model, input_size=(3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           9,408
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          36,864
       BatchNorm2d-6           [-1, 64, 16, 16]             128
              ReLU-7           [-1, 64, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          36,864
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
       BasicBlock-11           [-1, 64, 16, 16]               0
           Conv2d-12           [-1, 64, 16, 16]          36,864
      BatchNorm2d-13           [-1, 64, 16, 16]             128
             ReLU-14           [-1, 64,

In [0]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR

optimizer = optim.SGD(model.parameters(), lr=0.1,momentum=0.9 ) 
criterion = nn.CrossEntropyLoss()
scheduler = OneCycleLR(optimizer, 
                       max_lr = 0.8, 
                       total_steps=50, 
                       epochs=50,  
                       pct_start=0.2, 
                       anneal_strategy='linear', 
                       cycle_momentum=True, 
                       base_momentum=0.85, 
                       max_momentum=0.95, 
                       div_factor=10.0,final_div_factor =1e4)

In [16]:
train_test_loader.train_model(model,device,train_loader,test_loader,optimizer,criterion,50,scheduler,batch_scheduler=False,best_acc = 0,path = "tinymodel.pt")

EPOCH: 1 LR: 0.08




Test set: Average loss: 0.0076, Accuracy: 1720/33000 (5.21%)

accuracy increased, Saving model....
EPOCH: 2 LR: 0.16




Test set: Average loss: 0.0068, Accuracy: 3307/33000 (10.02%)

accuracy increased, Saving model....
EPOCH: 3 LR: 0.24




Test set: Average loss: 0.0064, Accuracy: 4549/33000 (13.78%)

accuracy increased, Saving model....
EPOCH: 4 LR: 0.32




Test set: Average loss: 0.0061, Accuracy: 5571/33000 (16.88%)

accuracy increased, Saving model....
EPOCH: 5 LR: 0.4


...



Test set: Average loss: 0.0059, Accuracy: 13193/33000 (39.98%)

accuracy increased, Saving model....
EPOCH: 47 LR: 0.06000739999999993




Test set: Average loss: 0.0059, Accuracy: 13305/33000 (40.32%)

accuracy increased, Saving model....
EPOCH: 48 LR: 0.04000760000000003




Test set: Average loss: 0.0059, Accuracy: 13390/33000 (40.58%)

accuracy increased, Saving model....
EPOCH: 49 LR: 0.02000780000000002




Test set: Average loss: 0.0059, Accuracy: 13515/33000 (40.95%)

accuracy increased, Saving model....
EPOCH: 50 LR: 8.000000000008e-06




Test set: Average loss: 0.0059, Accuracy: 13497/33000 (40.90%)

